# Polymer buildup and Hydrogen production in the ISIS Moderator

# COPY - add in the spatial stuff to this one 

- start off with splitting the dose rate so that the top is double the bottom - so 2/3 and 1/3 respectively
- add in the iterative stuff once i have the data and the volumes etc confirmed
- for the iterative stuff later, i believe everything (for now) other than dose is constant so can loop over a list of the dose values then add in other stuff like dwell time later

- may be useful to delete all the basic code with the functions and import them from the other notebook (like i did on steves neutron tools) as itll save a lot of space and be less bulky



### Radicals per Dose
* G value in Mol/J

\begin{aligned}
moles/MGy = \frac {G_{mol/J}}{1000} \times 1e6 \\ \\ 
radicals/MGy = moles \times 6.023e23
\end{aligned}

In [24]:
def moles_per_MGy(G_value):
    
    moles_per_g = G_value/1000    #since 1Gy = 1J/kg
    moles_per_MGy = moles_per_g * 1e6 
    
    return(moles_per_MGy)


In [25]:
def radicals_per_dose(moles):
    
    rads_per_dose = moles * 6.023e23
    
    return(rads_per_dose)

In [26]:
moles = moles_per_MGy(0.66e-6)
print (moles, 'mol/g')
print (radicals_per_dose(moles), 'radicals/g.MGy')

0.00066 mol/g
3.97518e+20 radicals/g.MGy


## Splitting the moderator so the top half is double the dose of the bottom
- So top half gets 2/300 bottom gets 1/300 MGy/s
- Dont need to iterate for this one as theres only two vals - repeat and add them together

In [27]:
#importing functions to save space and repetition
from Polymer_Base_code import vol_of_H_a
from Polymer_Base_code import vol_of_H_b
from Polymer_Base_code import dose_rate_conversion
from Polymer_Base_code import g_val_conversion
from Polymer_Base_code import rate_of_pol_form
from Polymer_Base_code import molecular_weight
#why is this printing?
#at least mr function works now

In [28]:
#for max chain length n = 20
Mr_20 = molecular_weight(20)
print(Mr_20)

280


### Evans Method

In [29]:
vol_top_a = vol_of_H_a (moles, 400, 2/300, 0.43)
vol_bottom_a = vol_of_H_a (moles, 400, 1/300, 0.43)
vol_total_a = vol_top_a + vol_bottom_a


print('top volume a =', vol_top_a, 'cm3/s')
print('bottom volume a =', vol_bottom_a, 'cm3/s')
print('total volume a =', vol_total_a, 'cm3/s')

top volume a = 16.95232 cm3/s
bottom volume a = 8.47616 cm3/s
total volume a = 25.42848 cm3/s


### Uni of Missouri Method

In [30]:
top_dose = dose_rate_conversion(2/300, 0.1696)
bottom_dose = dose_rate_conversion(1/300, 0.1696)

new_g_val = g_val_conversion(0.66e-6)

vol_top_b = vol_of_H_b(top_dose, new_g_val)
vol_bottom_b = vol_of_H_b(bottom_dose, new_g_val)
vol_total_b = vol_top_b + vol_bottom_b

print('top volume b =', vol_top_b, 'cm3/s')
print('bottom volume b =', vol_bottom_b, 'cm3/s')
print('total volume b =', vol_total_b, 'cm3/s')


top volume b = 16.715775999999998 cm3/s
bottom volume b = 8.357887999999999 cm3/s
total volume b = 25.073663999999997 cm3/s


### Polymer production rate

In [31]:
polymer_top = rate_of_pol_form(0.22e-6, Mr_20, 2/300, 169.6)
polymer_bottom = rate_of_pol_form(0.22e-6, Mr_20, 1/300, 169.6)
polymer_total = polymer_top + polymer_bottom

print('top polymer rate =', polymer_top, 'g polymer/s')
print('bottom polymer rate =', polymer_bottom, 'g polymer/s')
print('total polymer rate =', polymer_total, 'g polymer/s')


top polymer rate = 0.06964906666666668 g polymer/s
bottom polymer rate = 0.03482453333333334 g polymer/s
total polymer rate = 0.10447360000000001 g polymer/s


## Add in the code for the data here
- this is where the iteration for the dose comes in
- still not too sure with the dwell times as im not sure how time affects the rate of production or how to implement it
- All assuming max chain length of n=20 for now

### TS1 Decoupled Dual Foil Moderator
- Data is in Watts not MGy/s (or is it Watts/s?) - check with Steve
- also probably best to do it for both normal and half/half (assuming vapour only isnt relevant? or is it??)
- put the data into an array then iterate over it and add them
- again taking mr to be 300 - add in Mr_20 when import is fixed - then change to whatever distribution thingy we figure out 

Regions are:
- Al can
- Gd poison and Al support
- Methane regions

Respectively (not sure how to label each result individually)

In [32]:
import numpy as np

#### Function for conversion of Watts to MGy/s

In [33]:
def watts_to_MGy_per_sec(watts):
    
    new_dose = watts * (1e-6/0.1696)
    
    return(new_dose)

#### Normal

In [34]:
heat_load_normal_watts = [62.5, 5.3, 95.1]
heat_load_normal = []

for x in heat_load_normal_watts:
    
    conversion = watts_to_MGy_per_sec(x)
    heat_load_normal.append(conversion)

print('Heat load for normal operation', heat_load_normal, 'MGy/s')

Heat load for normal operation [0.0003685141509433962, 3.125e-05, 0.0005607311320754716] MGy/s


In [35]:
for x in heat_load_normal:
    
    pol_rate_normal = rate_of_pol_form(0.22e-6, Mr_20, x, 169.6)
    
    print(pol_rate_normal, 'g polymer/s')

0.0038499999999999997 g polymer/s
0.00032648000000000003 g polymer/s
0.005858159999999999 g polymer/s


#### Half liquid half vapour

In [36]:
heat_load_half_watts = [56.9, 3.7, 37.6]
heat_load_half = []

for x in heat_load_half_watts:
    
    conversion = watts_to_MGy_per_sec(x)
    heat_load_half.append(conversion)

print('Heat load for half liquid half vapour', heat_load_half, 'MGy/s')

Heat load for half liquid half vapour [0.0003354952830188679, 2.1816037735849055e-05, 0.00022169811320754717] MGy/s


In [37]:
for x in heat_load_half:
    
    pol_rate_half = rate_of_pol_form(0.22e-6, Mr_20, x, 169.6)
    
    print(pol_rate_half, 'g polymer/s')

0.00350504 g polymer/s
0.00022792 g polymer/s
0.0023161600000000003 g polymer/s


#### Vapour only

In [38]:
heat_load_vapour_watts = [54.0, 3.1, 0.4]
heat_load_vapour = []

for x in heat_load_vapour_watts:
    
    conversion = watts_to_MGy_per_sec(x)
    heat_load_vapour.append(conversion)

print('Heat load for vapour only', heat_load_vapour, 'MGy/s')

Heat load for vapour only [0.00031839622641509433, 1.8278301886792454e-05, 2.358490566037736e-06] MGy/s


In [39]:
for x in heat_load_vapour:
    
    pol_rate_vapour = rate_of_pol_form(0.22e-6, Mr_20, x, 169.6)
    
    print(pol_rate_vapour, 'g polymer/s')

0.0033264 g polymer/s
0.00019096000000000003 g polymer/s
2.464e-05 g polymer/s


#### Empty 
(not really sure if ive gotta do this one but ill do it for good luck anyway)

In [40]:
heat_load_empty_watts = [55.6, 2.9, 0.0]
heat_load_empty = []

for x in heat_load_empty_watts:
    
    conversion = watts_to_MGy_per_sec(x)
    heat_load_empty.append(conversion)

print('Heat load for empty moderator', heat_load_empty, 'MGy/s')

Heat load for empty moderator [0.0003278301886792453, 1.7099056603773582e-05, 0.0] MGy/s


In [41]:
for x in heat_load_vapour:
    
    pol_rate_vapour = rate_of_pol_form(0.22e-6, Mr_20, x, 169.6)
    
    print(pol_rate_vapour, 'g polymer/s')

0.0033264 g polymer/s
0.00019096000000000003 g polymer/s
2.464e-05 g polymer/s


### TS1 Decoupled Single Foil Moderator
- again in watts
- only nominal config and empty data due to modelling difficulties withtemperature dependant variables and timestep
- also includes errors (probably useful to add those in late but not too sure how yet)

Regions are:
- total in Al can
- total in Al pipe
- Gd poison and its Al support
- Methane regions

Respectively (not sure how to label each result individually)

#### Nominal Configuration

In [42]:
heat_load_nominal_watts = [37.9, 1.57, 3.0, 70.5]
heat_load_nominal = []

for x in heat_load_nominal_watts:
    
    conversion = watts_to_MGy_per_sec(x)
    heat_load_nominal.append(conversion)

print('Heat load for nominal configuration', heat_load_nominal, 'MGy/s')

Heat load for nominal configuration [0.00022346698113207545, 9.257075471698113e-06, 1.7688679245283016e-05, 0.0004156839622641509] MGy/s


In [43]:
for x in heat_load_nominal:
    
    pol_rate_nominal = rate_of_pol_form(0.22e-6, Mr_20, x, 169.6)
    
    print(pol_rate_nominal, 'g polymer/s')

0.00233464 g polymer/s
9.6712e-05 g polymer/s
0.00018479999999999997 g polymer/s
0.0043428 g polymer/s


#### Empty Moderator

In [44]:
heat_load_empty_single_watts = [37.9, 1.57, 3.0, 70.5]
heat_load_empty_single = []

for x in heat_load_empty_single_watts:
    
    conversion = watts_to_MGy_per_sec(x)
    heat_load_empty_single.append(conversion)

print('Heat load for empty moderator', heat_load_empty_single, 'MGy/s')

Heat load for empty moderator [0.00022346698113207545, 9.257075471698113e-06, 1.7688679245283016e-05, 0.0004156839622641509] MGy/s


In [45]:
for x in heat_load_empty_single:
    
    pol_rate_empty_single = rate_of_pol_form(0.22e-6, Mr_20, x, 169.6)
    
    print(pol_rate_empty_single, 'g polymer/s')

0.00233464 g polymer/s
9.6712e-05 g polymer/s
0.00018479999999999997 g polymer/s
0.0043428 g polymer/s
